In [1]:
import time
import datetime

class Equipo:
    def __init__(self, id_club, nombre):
        self.id_club = id_club
        self.nombre = nombre

    def __str__(self):
        return "%s - %s" % (self.id_club, self.nombre)


class Partido:
    def __init__(self, id_partido, temporada, division, jornada, local,
                 visitante, goles_local, goles_visitante, fecha):
        self.idPartido = id_partido
        self.temporada = temporada
        self.division = division
        self.jornada = jornada
        self.local = local
        self.visitante = visitante
        self.golesLocal = goles_local
        self.golesVisitante = goles_visitante
        self.fecha = fecha
        self.timestamp = int(time.mktime(
            datetime.datetime.strptime(fecha, "%d/%m/%Y").timetuple()))

    def __str__(self):
        return "%s::%s::%s::%s::%s::%s::%s::%s::%s::%s" \
               % (self.idPartido, self.temporada, self.division, self.jornada, self.local, self.visitante,
                  self.golesLocal, self.golesVisitante, self.fecha, self.timestamp)


In [2]:
# De la web: http://www.resultados-futbol.com/
# Para los resultados de la temporada 2020-2021
TEMPORADA_2021 = "2020-21"
URL_PRIMERA_2021 = "http://www.resultados-futbol.com/primera/grupo1/jornada%s"
URL_SEGUNDA_2021 = "http://www.resultados-futbol.com/segunda/grupo1/jornada%s"
MAX_JORNADAS_1 = 38   #38
MAX_JORNADAS_2 = 42   #42

In [3]:
from bs4 import BeautifulSoup
import requests
import re

# Guardo los partidos de futbol con un id
partidos = dict()

# Contador de Partidos
contador = 0

# Funcion para sustituir el nombre de los equipos y unificarlos
def replace_equipos(equipo):
    equipo = equipo.replace('Deportivo de La Coruña', 'Deportivo')
    equipo = equipo.replace('Barcelona Atletic', 'Barcelona B')    
    equipo = equipo.replace('Atletico-Madrid', 'Atletico de Madrid')
    equipo = equipo.replace('Real-Sociedad', 'Real Sociedad')
    equipo = equipo.replace('Celta', 'Celta de Vigo')
    equipo = equipo.replace('Athletic Club-Bilbao', 'Athletic Club')
    equipo = equipo.replace('Athletic-Bilbao', 'Athletic Club')
    equipo = equipo.replace('Real Betis', 'Betis')
    equipo = equipo.replace('Zaragoza', 'Real Zaragoza')
    equipo = equipo.replace('Recreativo', 'Recreativo de Huelva')
    equipo = equipo.replace('Sporting de Gijon-Gijon', 'Sporting de Gijon')
    equipo = equipo.replace('Sporting-Gijon', 'Sporting de Gijon')
    equipo = equipo.replace('Racing', 'Racing de Santander')
    equipo = equipo.replace('Real-Madrid', 'Real Madrid')
    equipo = equipo.replace('Valencia-Cf', 'Valencia')
    equipo = equipo.replace('Ud-Palmas', 'Las Palmas')
    equipo = equipo.replace('Gimnastic-Tarragona', 'Gimnastic de Tarragona')
    equipo = equipo.replace('Ucam-Murcia-C-F', 'UCAM Murcia')
    equipo = equipo.replace('Real-Real Zaragoza', 'Real Zaragoza')
    equipo = equipo.replace('Rayo-Vallecano', 'Rayo Vallecano')
    equipo = equipo.replace('Ad-Alcorcon', 'Alcorcon')
    equipo = equipo.replace('Real-Oviedo', 'Real Oviedo')
    equipo = equipo.replace('Sevilla-B', 'Sevilla Atletico')
    equipo = equipo.replace('Girona-Fc', 'Girona')

    return equipo


# Obtener la fecha del partido de futbol
def get_fecha_partido(fecha_sucia):
    charts_remove = ['[', '\\n', '\\t']
    fecha = re.sub(r'[<](/)?td[^>]*[>]', '', fecha_sucia).translate(None, ''.join(charts_remove))
    fecha = fecha.split('<br/>')[0].strip()
    fecha = fecha.split(' ')
    meses = {'Ene': '01', 'Feb': '02', 'Mar': '03', 'Abr': '04', 'May': '05', 'Jun': '06', 'Jul': '07', 'Ago': '08',
             'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dic': '12', 'Oc': '10', 'Ee': '01', 'Ju': '06'}
    fecha = "%s/%s/20%s" % (fecha[0], meses[fecha[1]], fecha[2])

    return fecha


# Obtener el nombre del equipo de futbol
def get_equipo(equipo_sucio):
    club = equipo_sucio.find_all('a')[1].attrs['href'].replace('/', '')
    return replace_equipos(club.strip())


# Obtener el resultado
def get_resultado(resultado_sucio):
    resultado = resultado_sucio.find('a').text
    resultado = re.sub(r'[<](/)?a[^>]*[>]', '', resultado).replace('</a>', '').strip()
    if 'x' in resultado:
        return None
    else:
        return resultado.split('-')


# Obtener los datos de cada partido de futbol
def get_partido(tr_partido):
    soup_tr = BeautifulSoup(tr_partido, "html.parser")

    fecha = get_fecha_partido(str(soup_tr.find_all('td', {'class': 'fecha'})))
    local = get_equipo(BeautifulSoup(str(soup_tr.find_all('td', {'class': 'equipo1'}))))
    visitante = get_equipo(BeautifulSoup(str(soup_tr.find_all('td', {'class': 'equipo2'}))))
    resultado = get_resultado(BeautifulSoup(str(soup_tr.find_all('td', {'class': 'rstd'}))))

    return {
        "local": local,
        "visitante": visitante,
        "gLocal": resultado[0],
        "gVisitante": resultado[1],
        "fecha": fecha
    }


# Obtener un listado con los partidos de Futbol
def find_partidos(tabla_partidos):
    partidos_jornada = list()
    soup_tabla = BeautifulSoup(tabla_partidos, "html.parser")
    tr_partidos = soup_tabla.find_all('tr', {'class': 'vevent'})

    for tr in tr_partidos:
        try:
            partidos_jornada.append(get_partido(str(tr)))
        except Exception as e:
            print (e)
    return partidos_jornada


# Obtener un diccionario con los partidos de futbol obtenidos de esta temporada
def get_partidos(contador):
    
    # Obtengo los partidos de primera division
    for i in range(1, MAX_JORNADAS_1):
        # Construyo las URLs
        url = URL_PRIMERA_2021 % i
        print ("Procesando %s" % url)

        # Realizo las peticiones a las URLs
        req_primera = requests.get(url)

        # Paso la request a un objeto BeautifulSoup
        soup_primera = BeautifulSoup(req_primera.text, "html.parser")

        # Obtengo la tabla con los resultado de los partidos de la jornada
        tabla_partidos = str(soup_primera.find('table', {'id': 'tabla1'}))
        partidos_jornada = find_partidos(tabla_partidos)
        for part in partidos_jornada:
            contador += 1
            partidos[contador] = Partido(contador, TEMPORADA_2021, 1, i,
                                         part['local'], part['visitante'],
                                         part['gLocal'], part['gVisitante'],
                                         part['fecha'])

    # Obtener los partidos de segunda division
    for i in range(1, MAX_JORNADAS_2 + 1):
        # Construyo las URLs
        url = URL_SEGUNDA_2021 % i
        print ("Procesando %s" % url)

        # Realizo las peticiones a las URLs
        req_segunda = requests.get(url)

        # Paso la request a un objeto BeautifulSoup
        soup_segunda = BeautifulSoup(req_segunda.text, "html.parser")

        # Obtengo la tabla con los resultado de los partidos de la jornada
        tabla_partidos = str(soup_segunda.find('table', {'id': 'tabla1'}))
        partidos_jornada = find_partidos(tabla_partidos)
        for part in partidos_jornada:
            contador += 1
            partidos[contador] = Partido(contador, TEMPORADA_2021, 2, i, part['local'], part['visitante'],
                                         part['gLocal'], part['gVisitante'], part['fecha'])

    return partidos

# Devuelve el valor del contador
def get_contador():
    return contador

In [4]:
# Obtener los partidos de futbol de la temporada presente
partidos_2020_21 = get_partidos(get_contador())

# Abrir archivo con el histórico de resultados y escribir los nuevos
fichero = open('data_7071_1920.txt', 'a')
for value in partidos_2020_21.values():
    fichero.write('%s\n' % str(value))

fichero.close()


Procesando http://www.resultados-futbol.com/primera/grupo1/jornada1
list index out of range
Procesando http://www.resultados-futbol.com/primera/grupo1/jornada2
list index out of range
list index out of range
Procesando http://www.resultados-futbol.com/primera/grupo1/jornada3
Procesando http://www.resultados-futbol.com/primera/grupo1/jornada4
Procesando http://www.resultados-futbol.com/primera/grupo1/jornada5
Procesando http://www.resultados-futbol.com/primera/grupo1/jornada6
Procesando http://www.resultados-futbol.com/primera/grupo1/jornada7
Procesando http://www.resultados-futbol.com/primera/grupo1/jornada8
Procesando http://www.resultados-futbol.com/primera/grupo1/jornada9
Procesando http://www.resultados-futbol.com/primera/grupo1/jornada10
Procesando http://www.resultados-futbol.com/primera/grupo1/jornada11
Procesando http://www.resultados-futbol.com/primera/grupo1/jornada12
Procesando http://www.resultados-futbol.com/primera/grupo1/jornada13
Procesando http://www.resultados-futbol.

'NoneType' object has no attribute '__getitem__'
'NoneType' object has no attribute '__getitem__'
'NoneType' object has no attribute '__getitem__'
'NoneType' object has no attribute '__getitem__'
'NoneType' object has no attribute '__getitem__'
'NoneType' object has no attribute '__getitem__'
'NoneType' object has no attribute '__getitem__'
'NoneType' object has no attribute '__getitem__'
'NoneType' object has no attribute '__getitem__'
'NoneType' object has no attribute '__getitem__'
'NoneType' object has no attribute '__getitem__'


In [6]:
# Sustituir el nombre de los equipos y unificarlos
data.EquipoLocal = data.EquipoLocal.replace('Espanyol', 'Espanol')
data.EquipoLocal = data.EquipoLocal.replace('Athletic Club', 'Atletico de Bilbao')
data.EquipoLocal = data.EquipoLocal.replace('Atletico Madrileno', 'Atletico de Madrid')
data.EquipoLocal = data.EquipoLocal.replace('Burgos CF', 'Burgos')
data.EquipoLocal = data.EquipoLocal.replace('Extremadura-Ud', 'Extremadura')
data.EquipoLocal = data.EquipoLocal.replace('Gimnastico de Tarragona', 'Gimnastic de Tarragona')
data.EquipoLocal = data.EquipoLocal.replace('CD Malaga', 'Malaga')
data.EquipoLocal = data.EquipoLocal.replace('Ponferradina-Sd', 'Ponferradina')
data.EquipoLocal = data.EquipoLocal.replace('Cd-Castellon', 'Castellon')
data.EquipoLocal = data.EquipoLocal.replace('Ud-Logrones', 'Logrones')  
data.EquipoLocal = data.EquipoLocal.replace('Real Oviedo', 'Oviedo') 

data.EquipoVisitante = data.EquipoVisitante.replace('Espanyol', 'Espanol')
data.EquipoVisitante = data.EquipoVisitante.replace('Athletic Club', 'Atletico de Bilbao')
data.EquipoVisitante = data.EquipoVisitante.replace('Atletico Madrileno', 'Atletico de Madrid')
data.EquipoVisitante = data.EquipoVisitante.replace('Burgos CF', 'Burgos')
data.EquipoVisitante = data.EquipoVisitante.replace('Extremadura-Ud', 'Extremadura')
data.EquipoVisitante = data.EquipoVisitante.replace('Gimnastico de Tarragona', 'Gimnastic de Tarragona')
data.EquipoVisitante = data.EquipoVisitante.replace('Ponferradina-Sd', 'Ponferradina')
data.EquipoVisitante = data.EquipoVisitante.replace('Cd-Castellon', 'Castellon')
data.EquipoVisitante = data.EquipoVisitante.replace('Ud-Logrones', 'Logrones')
data.EquipoVisitante = data.EquipoVisitante.replace('Real Oviedo', 'Oviedo')

In [7]:
import pandas as pd

# Guarda el archivo
data = pd.read_csv('data_7071_1920.txt', sep="::")